In [2]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Embedding, LSTM ,GRU
from keras.layers.embeddings import  Embedding
from keras.models import Sequential
from keras.initializers import Constant
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


C:\Users\Sergio\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
dt= pd.read_csv('train_tweet.csv')

In [4]:
#Transformar la columna de string a int 
dt['polarity_number']= np.NaN
dt.loc[dt['polarity']=='P', 'polarity_number' ]= 1
dt.loc[dt['polarity']=='N', 'polarity_number' ]= 0
dt.loc[dt['polarity']=='NEU', 'polarity_number' ]= np.NAN
dt.loc[dt['polarity']=='NONE', 'polarity_number' ]= np.NAN

In [5]:
#Eliminar las columnas que contienen nulos 
dt = dt.dropna()

In [6]:
#declarar las variables de entrenamiento
sentences= dt['content'].values
y= dt['polarity_number'].values

In [7]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences,
                                                                   y, test_size=0.25, random_state=1000)

In [8]:
review_lines = []
lines= list(sentences)

for line in lines:
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words= [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('spanish'))
    words= [w for w in words if not w in stop_words]
    review_lines.append(words)

In [9]:
review_lines

[['myendlesshazza',
  'puto',
  'mal',
  'escribo',
  'b',
  'sigo',
  'surrando',
  'help',
  'quedado',
  'raro',
  'cómetelo',
  'ahí',
  'jajajaja'],
 ['jajajaja', 'd', 'mucha', 'gente', 'seguro', 'puedo', 'melena', 'muero'],
 ['quiero', 'mogollón', 'sobretodo', 'rápido', 'contesta', 'wasaps'],
 ['vale',
  'visto',
  'tia',
  'bebiendose',
  'regla',
  'hs',
  'dado',
  'muchs',
  'grima'],
 ['yulianpoe', 'ah', 'supuesto', 'solo', 'incluyo', 'entendido', 'mal'],
 ['terminado',
  'lamentablemente',
  'arriendo',
  'ganancias',
  'pueblo',
  'brasileño',
  'penuria',
  'espera',
  'suerte',
  'solidaridad'],
 ['siiii', 'super', 'gracioso', 'teniamos', 'habernos', 'sacado', 'foto'],
 ['toniend',
  'seria',
  'mejor',
  'dejasen',
  'emitir',
  'basura',
  'evolucionar',
  'bien'],
 ['mandaria', 'comprarte', 'burro', 'creo', 'tiendas', 'abiertas', 'ahora'],
 ['mg', 'pongo', 'adjetivo', 'super', 'repelente', 'nombre'],
 ['hywzz', 'voz', 'maría', 'final', 'mata', 'jajajsjajajaj', 'quilla

In [10]:
len(review_lines)

736

In [11]:
max_length = max([len(s.split())for s in sentences])

max_length

30

In [12]:
embeddings_index = {}
with open('glove-sbwc.i25.vec',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

In [13]:
len(embeddings_index)

855381

In [14]:
tokenizer= Tokenizer()

tokenizer.fit_on_texts(review_lines)

sequences= tokenizer.texts_to_sequences(review_lines)

word_index= tokenizer.word_index

review_pad= pad_sequences(sequences, maxlen=max_length)

review_pad.shape,y.shape

((736, 30), (736,))

In [15]:
EMBEDDING_DIM = 300

In [16]:
num_words = len(word_index) +1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embeddings_vector = embeddings_index.get(word)
    if embeddings_vector is not None:
        embedding_matrix[i]= embeddings_vector

In [17]:
print(num_words)

3471


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Embedding , LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

In [19]:
model= Sequential()
embedding_layer = Embedding(num_words,
                           EMBEDDING_DIM,
                           embeddings_initializer = Constant(embedding_matrix),
                           input_length = max_length,
                           trainable= False)
model.add(embedding_layer)

model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           1041300   
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                31968     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,073,301
Trainable params: 32,001
Non-trainable params: 1,041,300
_________________________________________________________________


In [20]:
sentences_train, sentences_test, y_train, y_test = train_test_split(review_pad,
                                                                   y, test_size=0.25, random_state=1000)

In [21]:
sentences_train.shape, sentences_test.shape

((552, 30), (184, 30))

In [22]:
sentences_train[0].shape

(30,)

In [23]:
model.fit(sentences_train, y_train, batch_size=128, epochs=25, validation_data=(sentences_test,y_test),verbose=2)

Train on 552 samples, validate on 184 samples
Epoch 1/25
 - 2s - loss: 0.6910 - acc: 0.5290 - val_loss: 0.6789 - val_acc: 0.5924
Epoch 2/25
 - 0s - loss: 0.6523 - acc: 0.6196 - val_loss: 0.6643 - val_acc: 0.5978
Epoch 3/25
 - 0s - loss: 0.6313 - acc: 0.6540 - val_loss: 0.6570 - val_acc: 0.5978
Epoch 4/25
 - 0s - loss: 0.6052 - acc: 0.6920 - val_loss: 0.6519 - val_acc: 0.6087
Epoch 5/25
 - 0s - loss: 0.5753 - acc: 0.7319 - val_loss: 0.6457 - val_acc: 0.6250
Epoch 6/25
 - 0s - loss: 0.5607 - acc: 0.7428 - val_loss: 0.6430 - val_acc: 0.6196
Epoch 7/25
 - 0s - loss: 0.5360 - acc: 0.7681 - val_loss: 0.6395 - val_acc: 0.6250
Epoch 8/25
 - 0s - loss: 0.5160 - acc: 0.7844 - val_loss: 0.6361 - val_acc: 0.6304
Epoch 9/25
 - 0s - loss: 0.4988 - acc: 0.7844 - val_loss: 0.6352 - val_acc: 0.6304
Epoch 10/25
 - 0s - loss: 0.4775 - acc: 0.7899 - val_loss: 0.6360 - val_acc: 0.6413
Epoch 11/25
 - 0s - loss: 0.4539 - acc: 0.8170 - val_loss: 0.6351 - val_acc: 0.6304
Epoch 12/25
 - 0s - loss: 0.4274 - acc:

In [24]:
loss, accuracy = model.evaluate(sentences_test, y_test, batch_size=128)
print('Accuracy: %f' % (accuracy*100))

184/184 [==============================] - 0s 450us/step
Accuracy: 71.195652


In [25]:
prueba= sentences_test[0:1]

In [26]:
prueba

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 891, 892,   2, 423, 261, 893,  27,
         95,  47, 121,  40]])

In [27]:
model.predict(x= prueba)

array([[0.9960145]], dtype=float32)

In [28]:
texto_prueba= 'mal fatal lo peor'

In [29]:
texto_prueba= texto_prueba.split()

In [30]:
texto_prueba= tokenizer.texts_to_sequences(texto_prueba)

In [31]:
texto_prueba= pad_sequences(texto_prueba, maxlen= max_length)

In [32]:
texto_prueba.shape

(4, 30)

In [33]:
model.predict(x=texto_prueba).mean()

0.15754372

In [34]:
import pickle

In [35]:
filename = 'model_predict_keras.sav'
pickle.dump(model, open(filename, 'wb'))

In [46]:
texto_prueba_01= 'bien, bueno, perfecto'

In [47]:
texto_prueba_01= texto_prueba_01.split()

In [48]:
texto_prueba_01= tokenizer.texts_to_sequences(texto_prueba_01)

In [49]:
texto_prueba_01= pad_sequences(texto_prueba_01, maxlen= max_length)

In [50]:
texto_prueba_01.shape

(3, 30)

In [51]:
model.predict(x=texto_prueba_01).mean()

0.5296451